Calculate THQ (Monte Carlo) 

计算THQ(蒙特卡洛)

In [1]:
import sys
sys.path.append('..')
import pandas as pd
import numpy as np
import myfunction as mf
path_data_raw = "C:/Users/dell/OneDrive/file/"
path_country_nc = "C:/Users/dell/OneDrive/file/nc"
path_country_csv = "C:/Users/dell/OneDrive/file/csv/"
path_one_spdb = 'C:/Users/dell/OneDrive/file/SPDB/'
drive_letter = 'E:'

path_pre = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/part0_treat/pretreatment/"
path_match = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/part0_treat/match/"
path_semdata = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/part0_treat/semdata/"


path_2_preanalysis_data = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/part2_sem/preanalysis/"
path_2_preanalysis_fig = drive_letter + "/wyy/code_project/running_outcome/final_fig/SPDB/part2_sem/preanalysis/"


path_3_sw_forecast = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/part3_forecast/sw_forecast/"
path_temp = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/temp/"

path_4_risk_assess = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/part4_assess/"
path_4_risk_assess_grid = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/part4_assess/grid/"

path_1_describe_fig_fish = drive_letter + "/wyy/code_project/running_outcome/final_fig/SPDB/part1_describe/fish_diff/"
path_1_describe_global_map = drive_letter + "/wyy/code_project/running_outcome/final_fig/SPDB/part1_describe/global_map/"

mark_num = "25"
meta_name = "meta_data.csv"

list_pfas =['PFOA', 'PFNA', 'PFDA', 'PFUnDA','PFDoDA','PFTrDA', 'PFTeDA', 'PFHxS', 'PFOS', 'FOSA', 'PFBA', 'PFPeA', 'PFHxA', 'PFHpA','PFBS']
list_pfas_lc = ['PFOA', 'PFNA', 'PFDA', 'PFUnDA','PFDoDA','PFTrDA', 'PFTeDA', 'PFHxS', 'PFOS', 'FOSA']
list_pfas_sc = ['PFBA', 'PFPeA', 'PFHxA', 'PFHpA','PFBS']
list_color = ["#4d8cbf", "#4f9c8b", "#555c6c", "#d77563", "#7d84a8", "#84aeb8", "#c3473b", "#89756d","#ffb3cc","#9a7ebf","#ffddb8", "#c4eaff", "#d1c6ff", "#c2ffbf", "#f5f5b0",]

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import truncnorm, triang
import os
import warnings
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)

def truncated_normal(mean, cv, lower, upper):
    std = mean * cv
    a, b = (lower - mean) / std, (upper - mean) / std
    return truncnorm(a, b, loc=mean, scale=std)

def generate_consume(consume_series, cv=0.05):
    result = np.zeros(len(consume_series))
    non_zero_mask = consume_series > 0
    
    if non_zero_mask.any():
        non_zero_values = consume_series[non_zero_mask]
        generated_values = truncated_normal(
            non_zero_values, 
            cv, 
            non_zero_values * 0.8, 
            non_zero_values * 1.2
        ).rvs()
        result[non_zero_mask] = generated_values
    
    return result

def calculate_thq(tc, fir, ef, ed, rfd, bw, at):
    return (tc * fir * ef * ed) / (rfd * bw * at) 


In [3]:

df_weight = pd.read_csv(path_4_risk_assess + 'hi/weight.csv')
df_life = pd.read_csv(path_4_risk_assess + 'hi/life_ex.csv')
df_rfd = pd.read_csv(path_4_risk_assess + 'hi/rfd.csv')

pfas_list = ['PFBS', 'PFDA', 'PFDoDA', 'PFHpA', 'PFHxA', 'PFHxS', 'PFNA', 'PFOA',
               'PFOS', 'PFPeA', 'PFTeDA', 'PFTrDA', 'PFUnDA']
n_simulations = 1000

source = 'lr'
for pfas in pfas_list:
    df_pfas = pd.read_csv(path_4_risk_assess + f'pfas/{source}_{pfas}.csv')
    df_pfas = df_pfas[df_pfas[pfas + '_min'].notna()]
    df_pfas = df_pfas.merge(df_weight, on='country_id', how='left')
    df_pfas = df_pfas.merge(df_life, on=['country_id', 'year'], how='left')
    rfd_data = df_rfd[df_rfd['PFAS'] == pfas].iloc[0]
    results = df_pfas[['lon', 'lat', 'year']].copy()

    for i in range(n_simulations):
        tc_column = np.random.choice([f'{pfas}_min', f'{pfas}_max', f'{pfas}_median'])
        tc = df_pfas[tc_column]/1000
        if rfd_data['min'] == rfd_data['max'] == rfd_data['median']:
            rfd = rfd_data['median'] / 1000000
        else:
            min_val = rfd_data['min'] / 1000000
            max_val = rfd_data['max'] / 1000000
            median_val = rfd_data['median'] / 1000000
            c = (median_val - min_val) / (max_val - min_val)
            loc = min_val
            scale = max_val - min_val
            
            rfd = triang.rvs(c, loc=loc, scale=scale)
        bw = truncated_normal(df_pfas['weight'], 0.05, df_pfas['weight'] * 0.8, df_pfas['weight'] * 1.2).rvs()
        life_ex = truncated_normal(df_pfas['life_ex'], 0.05, df_pfas['life_ex'] * 0.8, df_pfas['life_ex'] * 1.2).rvs()

        ff_consume = generate_consume(df_pfas['ff_consume'])
        sf_consume = generate_consume(df_pfas['sf_consume'])
        ef = 365
        at = 365 * life_ex
        
        thq_ff = calculate_thq(tc, ff_consume, ef, life_ex, rfd, bw, at)
        thq_sf = calculate_thq(tc, sf_consume, ef, life_ex, rfd, bw, at)
        thq_fish = thq_ff + thq_sf
        results[f'thq_fish_{i}'] = thq_fish*0.71
    output_path = path_4_risk_assess + f'results/{source}_{pfas}_thq.csv'
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    results.to_csv(output_path, index=False)
print("Over！")

分析完成！


In [4]:

df_weight = pd.read_csv(path_4_risk_assess + 'hi/weight.csv')
df_life = pd.read_csv(path_4_risk_assess + 'hi/life_ex.csv')
df_rfd = pd.read_csv(path_4_risk_assess + 'hi/rfd.csv')
pfas_list = ['FOSA', 'PFBA', 'PFBS', 'PFDA', 'PFDoDA', 'PFHpA', 'PFHxA', 'PFHxS', 'PFNA', 'PFOA',
              'PFOS', 'PFPeA', 'PFTeDA', 'PFTrDA', 'PFUnDA']
n_simulations = 1000

source = 'sw'
for pfas in pfas_list:
    df_pfas = pd.read_csv(path_4_risk_assess + f'pfas/{source}_{pfas}.csv')
    df_pfas = df_pfas[df_pfas[pfas + '_min'].notna()]
    df_pfas = df_pfas.merge(df_weight, on='country_id', how='left')
    df_pfas = df_pfas.merge(df_life, on=['country_id', 'year'], how='left')
    df_pfas['sw_consume'] = 2*0.2*0.5
    rfd_data = df_rfd[df_rfd['PFAS'] == pfas].iloc[0]
    results = df_pfas[['lon', 'lat', 'year']].copy()
    for i in range(n_simulations):
        tc_column = np.random.choice([f'{pfas}_min', f'{pfas}_max', f'{pfas}_median'])
        tc = df_pfas[tc_column]/1000
        if rfd_data['min'] == rfd_data['max'] == rfd_data['median']:
            rfd = rfd_data['median'] / 1000000
        else:
            min_val = rfd_data['min'] / 1000000
            max_val = rfd_data['max'] / 1000000
            median_val = rfd_data['median'] / 1000000
            c = (median_val - min_val) / (max_val - min_val)
            loc = min_val
            scale = max_val - min_val
            
            rfd = triang.rvs(c, loc=loc, scale=scale)
        bw = truncated_normal(df_pfas['weight'], 0.05, df_pfas['weight'] * 0.8, df_pfas['weight'] * 1.2).rvs()
        life_ex = truncated_normal(df_pfas['life_ex'], 0.05, df_pfas['life_ex'] * 0.8, df_pfas['life_ex'] * 1.2).rvs()

        sw_consume  = generate_consume(df_pfas['sw_consume'])

        ef = 365
        at = 365 * life_ex

        thq_water = calculate_thq(tc, sw_consume, ef, life_ex, rfd, bw, at)
        results[f'thq_water_{i}'] = thq_water
    output_path = path_4_risk_assess + f'results/{source}_{pfas}_thq.csv'
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    results.to_csv(output_path, index=False)
print("Over！")
# 115min

分析完成！


In [5]:
import os
import pandas as pd
import glob

input_path = r'E:\wyy\code_project\running_outcome\final_data\SPDB\part4_assess\results'
output_path = r'E:\wyy\code_project\running_outcome\final_data\SPDB\part4_assess\thq'

if not os.path.exists(output_path):
    os.makedirs(output_path)
middle_parts = ['FOSA', 'PFBA', 'PFBS', 'PFDA', 'PFDoDA', 'PFHpA', 'PFHxA', 'PFHxS', 'PFNA', 'PFOA', 'PFOS', 'PFPeA', 'PFTeDA', 'PFTrDA', 'PFUnDA']

for prefix in ['lr_', 'sw_']:
    merged_df = None
    
    for middle in middle_parts:
        file_name = f"{prefix}{middle}_thq.csv"
        file_path = os.path.join(input_path, file_name)
        df = pd.read_csv(file_path)
        print(df[['lon', 'lat', 'year']].head(5))
        if merged_df is None:
            merged_df = df
        else:
            merged_df.iloc[:, 3:] += df.iloc[:, 3:]
        del df
    output_file = os.path.join(output_path, f"{prefix}merged_thq.csv")
    merged_df.to_csv(output_file, index=False)
    print(f"Merged file saved: {output_file}")

print("All files have been processed and merged.")

# 总共40min

    lon   lat  year
0 -69.0 -55.0  2000
1 -68.0 -55.0  2000
2 -71.0 -54.0  2000
3 -68.0 -54.0  2000
4 -69.0 -53.0  2000
    lon   lat  year
0 -69.0 -55.0  2000
1 -68.0 -55.0  2000
2 -71.0 -54.0  2000
3 -68.0 -54.0  2000
4 -69.0 -53.0  2000
    lon   lat  year
0 -69.0 -55.0  2000
1 -68.0 -55.0  2000
2 -71.0 -54.0  2000
3 -68.0 -54.0  2000
4 -69.0 -53.0  2000
    lon   lat  year
0 -69.0 -55.0  2000
1 -68.0 -55.0  2000
2 -71.0 -54.0  2000
3 -68.0 -54.0  2000
4 -69.0 -53.0  2000
    lon   lat  year
0 -69.0 -55.0  2000
1 -68.0 -55.0  2000
2 -71.0 -54.0  2000
3 -68.0 -54.0  2000
4 -69.0 -53.0  2000
    lon   lat  year
0 -69.0 -55.0  2000
1 -68.0 -55.0  2000
2 -71.0 -54.0  2000
3 -68.0 -54.0  2000
4 -69.0 -53.0  2000
    lon   lat  year
0 -69.0 -55.0  2000
1 -68.0 -55.0  2000
2 -71.0 -54.0  2000
3 -68.0 -54.0  2000
4 -69.0 -53.0  2000
    lon   lat  year
0 -69.0 -55.0  2000
1 -68.0 -55.0  2000
2 -71.0 -54.0  2000
3 -68.0 -54.0  2000
4 -69.0 -53.0  2000
    lon   lat  year
0 -69.0 -55.0  2000


In [6]:
import os
import pandas as pd
import glob
input_path = r'E:\wyy\code_project\running_outcome\final_data\SPDB\part4_assess\results'
output_path = r'E:\wyy\code_project\running_outcome\final_data\SPDB\part4_assess\thq'
if not os.path.exists(output_path):
    os.makedirs(output_path)
middle_parts = ['FOSA', 'PFBA', 'PFBS', 'PFDA', 'PFDoDA', 'PFHpA', 'PFHxA', 'PFHxS', 'PFNA', 'PFOA', 'PFOS', 'PFPeA', 'PFTeDA', 'PFTrDA', 'PFUnDA']
for prefix in ['lr_', 'sw_']:
    merged_df = None
    
    for middle in middle_parts:
        file_name = f"{prefix}{middle}_thq.csv"
        file_path = os.path.join(input_path, file_name)
        df = pd.read_csv(file_path)
        print(df[['lon', 'lat', 'year']].head(5))
        if merged_df is None:
            merged_df = df
        else:
            merged_df.iloc[:, 3:] += df.iloc[:, 3:]
        del df
    
    output_file = os.path.join(output_path, f"{prefix}merged_thq.csv")
    merged_df.to_csv(output_file, index=False)
    print(f"Merged file saved: {output_file}")

print("All files have been processed and merged.")

# 总共40min

    lon   lat  year
0 -69.0 -55.0  2000
1 -68.0 -55.0  2000
2 -71.0 -54.0  2000
3 -68.0 -54.0  2000
4 -69.0 -53.0  2000
    lon   lat  year
0 -69.0 -55.0  2000
1 -68.0 -55.0  2000
2 -71.0 -54.0  2000
3 -68.0 -54.0  2000
4 -69.0 -53.0  2000
    lon   lat  year
0 -69.0 -55.0  2000
1 -68.0 -55.0  2000
2 -71.0 -54.0  2000
3 -68.0 -54.0  2000
4 -69.0 -53.0  2000
    lon   lat  year
0 -69.0 -55.0  2000
1 -68.0 -55.0  2000
2 -71.0 -54.0  2000
3 -68.0 -54.0  2000
4 -69.0 -53.0  2000
    lon   lat  year
0 -69.0 -55.0  2000
1 -68.0 -55.0  2000
2 -71.0 -54.0  2000
3 -68.0 -54.0  2000
4 -69.0 -53.0  2000
    lon   lat  year
0 -69.0 -55.0  2000
1 -68.0 -55.0  2000
2 -71.0 -54.0  2000
3 -68.0 -54.0  2000
4 -69.0 -53.0  2000
    lon   lat  year
0 -69.0 -55.0  2000
1 -68.0 -55.0  2000
2 -71.0 -54.0  2000
3 -68.0 -54.0  2000
4 -69.0 -53.0  2000
    lon   lat  year
0 -69.0 -55.0  2000
1 -68.0 -55.0  2000
2 -71.0 -54.0  2000
3 -68.0 -54.0  2000
4 -69.0 -53.0  2000
    lon   lat  year
0 -69.0 -55.0  2000


In [8]:
input_path = r'E:\wyy\code_project\running_outcome\final_data\SPDB\part4_assess\thq'
file_name1 = 'lr_merged_thq.csv'
file_name2 = 'sw_merged_thq.csv'
df_lr = pd.read_csv(os.path.join(input_path, file_name1))
print(df_lr.dtypes)
df_sw = pd.read_csv(os.path.join(input_path, file_name2))
print(df_sw.dtypes)

lon             float64
lat             float64
year              int64
thq_fish_0      float64
thq_fish_1      float64
                 ...   
thq_fish_995    float64
thq_fish_996    float64
thq_fish_997    float64
thq_fish_998    float64
thq_fish_999    float64
Length: 1003, dtype: object
lon              float64
lat              float64
year               int64
thq_water_0      float64
thq_water_1      float64
                  ...   
thq_water_995    float64
thq_water_996    float64
thq_water_997    float64
thq_water_998    float64
thq_water_999    float64
Length: 1003, dtype: object


In [9]:
import pandas as pd
import os

input_path = r'E:\wyy\code_project\running_outcome\final_data\SPDB\part4_assess\thq'
file_name1 = 'lr_merged_thq.csv'
file_name2 = 'sw_merged_thq.csv'

df_lr = pd.read_csv(os.path.join(input_path, file_name1))
df_sw = pd.read_csv(os.path.join(input_path, file_name2))

assert (df_lr[['lon', 'lat', 'year']] == df_sw[['lon', 'lat', 'year']]).all().all(), "lon, lat, year columns do not match"
df_merged = df_lr[['lon', 'lat', 'year']]

for i in range(1000):
    df_merged[f'thq_{i}'] = df_lr[f'thq_fish_{i}'] + df_sw[f'thq_water_{i}']

output_file = 'merged_thq.csv'
df_merged.to_csv(os.path.join(input_path, output_file), index=False)

print(f"Merged data saved to {os.path.join(input_path, output_file)}")

Merged data saved to E:\wyy\code_project\running_outcome\final_data\SPDB\part4_assess\thq\merged_thq.csv
